In [35]:
! pip install -qU -r requirements.txt

In [58]:
from decouple import AutoConfig, config
from icecream import ic
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

In [4]:
config = AutoConfig(search_path='../notes-rag/')

LANGCHAIN_TRACING_V2 = config("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = config("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT = config("LANGCHAIN_PROJECT")
OPENAI_API_KEY = config("OPENAI_API_KEY")
lANGCHAIN_API_KEY = config("LANGCHAIN_API_KEY")
HF_TOKEN = config("HF_TOKEN")

In [47]:
loader = WebBaseLoader(web_path='https://docs.smith.langchain.com/prompt_engineering/concepts').load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-4o")
split_docs = text_splitter.split_documents(loader)

In [32]:
for doc in split_docs:
    print(doc.page_content)
    print("-----")

Concepts | 🦜️🛠️ LangSmith
-----
Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringConceptual GuideHow-to GuidesPlaygroundPromptsTutorialsOptimize a classifierDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions
-----
(LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referencePrompt EngineeringOn this pageConcepts
-----
Prompt engineering is one the core pillars of LangSmith.
While traditional software application are built by writing code, AI applications often involve a good amount of writing prompts.
We aim to make this as easy possible b

In [37]:
db = FAISS.from_documents(split_docs, embeddings)

In [41]:
db.similarity_search("difference between prompt and prompt template")

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'Prompt engineering is one the core pillars of LangSmith.', 'language': 'en'}, page_content='Prompts refer to the messages that are passed into the language model.\nPrompt Templates refer to a way of formatting information to get that prompt to hold the information that you want. Prompt templates\ncan include variables for few shot examples, outside context, or any other external data that is needed in your prompt.'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'Prompt engineering is one the core pillars of LangSmith.', 'language': 'en'}, page_content='or another domain expert.\nIt is important to have the proper tooling and infrastructure to support this cross-disciplinary building.\nPrompts vs Prompt Templates\u200b\nAlthough we often use

In [51]:
retriever = db.as_retriever() # converts a vector store to a retriever

In [48]:
prompt = ChatPromptTemplate.from_template(
        """
        Answer the following question based only on the provided context:
            <context>
                {context}
            </context>
        """
    )
document_chain = create_stuff_documents_chain(llm, prompt)

In [57]:
document_chain.invoke({
    "input": "difference between prompt and prompt template",
    "context": [Document(page_content="langchain is a good framework")]
})

'Based on the provided context, Langchain is described as a good framework.'

In [60]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [61]:
response = retrieval_chain.invoke({"input": "difference between prompt and prompt template"})

In [62]:
response

{'input': 'difference between prompt and prompt template',
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'Prompt engineering is one the core pillars of LangSmith.', 'language': 'en'}, page_content='Prompts refer to the messages that are passed into the language model.\nPrompt Templates refer to a way of formatting information to get that prompt to hold the information that you want. Prompt templates\ncan include variables for few shot examples, outside context, or any other external data that is needed in your prompt.'),
  Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'Prompt engineering is one the core pillars of LangSmith.', 'language': 'en'}, page_content='or another domain expert.\nIt is important to have the proper tooling and infrastructure to support this cross-discipli